In [2]:
%load_ext autoreload
%load_ext watermark
%autoreload 2
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,tensorflow,keras

Sun Feb 24 2019 

CPython 3.6.8
IPython 7.2.0

numpy 1.15.4
scipy 1.2.0
sklearn not installed
pandas 0.23.4
tensorflow 1.12.0
keras 2.2.4

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-43-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


Using TensorFlow backend.


In [3]:
# TODO visualise and summarise the corpus data

In [4]:
from data.preprocess import PreProcessor

data = PreProcessor(config=PreProcessor.DEFAULT_CONFIG,
                    data_dir='data/raw/r252-corpus-features/org/elasticsearch/action/admin/cluster/allocation/')

vocab = data.metadata['token_vocab']
processed = data.get_tensorise_data()


In [5]:
label_name = np.expand_dims(processed['name_tokens'], 2)

In [18]:
import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras import layers

from models.cnn_attention import ConvAttention

code_snippet = processed['body_tokens']
label_name = np.expand_dims(processed['name_tokens'], 2)

embedding_dim = 128
vocabulary_size = len(vocab)
max_chunk_length = data.config['max_chunk_length']

# TODO make the input a json file and parse it
batch_size = 1
k1 = 8
k2 = 8
w1 = 24
w2 = 29
w3 = 10
dropout_rate = 0.5

# Optimised hyperparameter are reported in page 5 of the paper

# define layers
main_input = layers.Input(shape=(max_chunk_length,),
                          batch_size=1,
                          dtype='int32', name='main_input')

cnn_layer = ConvAttention(vocabulary_size=vocabulary_size,
                          embedding_dim=embedding_dim,
                          max_chunk_length=max_chunk_length,
                          k1=k1,
                          k2=k2,
                          w1=w1,
                          w2=w2,
                          w3=w3,
                          dropout_rate=dropout_rate)

optimizer = keras.optimizers.Nadam()  # RMSprop with Nesterov momentum
loss_func = keras.losses.mean_squared_error

# define execution
cnn_output = cnn_layer(main_input)
model = keras.Model(inputs=[main_input], outputs=cnn_output)
model.compile(optimizer=optimizer,
              loss=loss_func,
              metrics=['accuracy'])
# fit the model
print("Code snippet len: {} label_name len: {}".format(len(code_snippet), len(label_name)))
history = model.fit(code_snippet, label_name, epochs=5, verbose=2, batch_size=1)


ConvAttention: Tokens shape = (1, 50, 128), h_t shape = (1, 8)
AttentionFeatures: C shape = (1, 50, 128), h_t shape = (1, 8)
AttentionFeatures: L_1 shape = (1, 50, 8)
AttentionFeatures: L_2 shape = (1, 50, 8)
AttentionFeatures: L_2 shape  after multiply = (1, 50, 8)
AttentionFeatures: L_feat shape = Tensor("conv_attention_8/attention_features_8/l2_normalize:0", shape=(1, 50, 8), dtype=float32)
ConvAttention: L_feat shape = (1, 50, 8)
AttentionWeights: l_feat shape = (1, 50, 8)
AttentionWeights: attention_weight shape = (1, 50, 1)
ConvAttention: n_hat shape = (1, 128)
ConvAttention: softmax shape = (128, 50, 1)
Code snippet len: 135 label_name len: 135


/home/samialab/anaconda3/envs/method-name-prediction/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5


 - 5s - loss: 1603.9984 - acc: 0.0013


Epoch 2/5


...

 - 4s - loss: 1603.9984 - acc: 0.0013


Epoch 4/5


 - 4s - loss: 1603.9984 - acc: 0.0013


Epoch 5/5


 - 4s - loss: 1603.9984 - acc: 0.0013


In [64]:
model.summary()
# # overfit and evaluate the model 
# loss, accuracy = model.evaluate(code_snippet, label_name, verbose=0)
# print('Accuracy: {}'.format(accuracy * 100))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (1, 50)              0                                            
__________________________________________________________________________________________________
embedding_33 (Embedding)        (1, 50, 128)         40832       main_input[0][0]                 
__________________________________________________________________________________________________
gru_33 (GRU)                    [(1, 8), (1, 8)]     3288        embedding_33[0][0]               
__________________________________________________________________________________________________
conv_attention_33 (ConvAttentio (1, 50, 1)           26529       embedding_33[0][0]               
                                                                 gru_33[0][1]                     
Total para

In [17]:
# translate prediction

from data.utils import translate_tokenized_array_to_list_words

# prediction = model.predict(code_snippet[5].reshape(1, -1))
# translate_tokenized_array_to_list_words(vocab, [0])
# print(prediction)
# vocab.get_name_for_id(317)
code_snippet[0, 0]

317